## 作业实现思路说明
利用python通过mongodb进行数据分析工作，工作重心在于前期数据整理和之后的查询分析。根据所学知识，首先需要进行数据准备工作，对读入的样本数据进行测试，检查数据规范性、是否存在不正常的数据。修复数据后整理成可插入的数据格式插入到mongo中，再做后续的各类分析，挖掘数据信息

### 键值组合模式分析
通过之前的课程学习，能够了解到所有提交的数据都可能存在一些并不规范的数据，为了能够统一的进行识别处理按照所学知识对所有tag标签的属性值进行收集检查。排查分析后的数据可知，k键值对中可能存在异常字符影响分析的标签属性有14个，没有进行归类定义的有8601个，其他基本都属于比较合理常见的键值，

In [1]:

# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs


"""
针对数据和官网介绍的信息数据结构说明，以及之前项目整理的正则表达式，分析出目前数据中能发现的几种情况，其中对问题数据和未归类数据都将进行具体数据的记录
生成问题数据文件，尝试过直接输出在页面，但是会影响加载速度，因此单独保存问题数字字段
"""

lower = re.compile(r'^([a-z]|_)*$')
upper = re.compile(r'^([A-Z]|_)*$')
mixed = re.compile(r'^([a-zA-Z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
splittime = ["1次分割", "2次分割", "3次分割", "4次分割"]
othercnt = []
problemcnt = []


def key_type(element, keys):
    if element.tag == "tag":
        key = element.attrib['k']
        keys=status_colon(key,keys)   
    return keys

def status(key):
    if problemchars.search(key):
        problemcnt.append(key)
        return "异常字符"
    elif lower.search(key):
        return "小写"
    elif upper.search(key):
        return "大写"
    elif mixed.search(key):
        return "大小写混合"
    else:
        othercnt.append(key)
        return "其他类型"

def status_colon(key,keys):
    keySplit = check_colon(key)
    if(len(keySplit)==1):
        process_status(splittime[0], keys)   
        keyStatus=status(key)        
        process_status(keyStatus, keys)
    else:
        keyStatus=splittime[len(keySplit)-1]
        process_status(keyStatus, keys)
        
    return keys

def check_colon(key):
    return re.split('\:', key)

def process_status(status, keys):
    if status in keys:
        keys[status] += 1
    else:
        keys[status] = 1
    return keys


def process_map(filename):
    keys = {}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
        element.clear()
    return keys

def test():
    keys = process_map('chicago.osm')
    pprint.pprint(keys)
    
    file_out = "problemchars-chicago.csv"
    data = []
    with codecs.open(file_out, "w", "utf-8") as fo:
        for line in problemcnt:
            fo.write(line+"\n")

    file_out = "othercntchars-chicago.csv"
    with codecs.open(file_out, "w", "utf-8") as fo:
        for line in othercnt:
            fo.write(line+"\n")
            
            
if __name__ == "__main__":
    test()
    

{'1次分割': 1617867,
 '2次分割': 3053318,
 '3次分割': 1487444,
 '其他类型': 8601,
 '大写': 1131,
 '大小写混合': 29,
 '小写': 1608092,
 '异常字符': 14}


#### 统计文件中出现过的所有标签数量和标签内attribute属性键值对的数量
设定数据整理思路需要了解大概的标签结构构成。通过结合官方文档说明以及样本数据的构成，设计对本次作业最有效的数据整理方式。
除了文件描述标签外，实际存在的标签可以分为 member，node，tag，relation，way，nd，标签次数记录在attrnum，每个标签的attribute分别记录，通过这种方式可以更好的规划到每个标签在后续过程中需要进行的数据处理判断逻辑

In [3]:

# -*- coding: utf-8 -*-
"""
首先从数据表中了解有多少个标签tag,以及每个标签下有多少个属性attr，后续在数据整理入库时作为结构参考
"""
import xml.etree.cElementTree as ET

def collect_tags(filename):
   
    tags={}
    
    
    for event, elem in ET.iterparse(filename):
        if event == 'end':
            tag = elem.tag
            #区分现在的字典表中是否有标签存在，并进行初始化或者计数
            if tag not in tags.keys():
                tags[tag]={}
                tags[tag]["attrnum"]=1
            else:
                tags[tag]["attrnum"]+=1
                
            #区分现在的标签字典表中是否有遍历的属性存在，并进行初始化或者计数
            attrkeys=elem.attrib            
            for key in attrkeys:          
                if(key not in tags[tag].keys()):
                    tags[tag][key]=1
                else:
                    tags[tag][key]+=1
        elem.clear() #通过测试发现如果不清空，内存不会释放
                
    return tags

##查组成要素数量
tags = collect_tags('chicago.osm')
pprint.pprint(tags)



{'bounds': {'attrnum': 1,
            'maxlat': 1,
            'maxlon': 1,
            'minlat': 1,
            'minlon': 1,
            'origin': 1},
 'member': {'attrnum': 43678, 'ref': 43678, 'role': 43678, 'type': 43678},
 'nd': {'attrnum': 9072027, 'ref': 9072027},
 'node': {'attrnum': 7679935,
          'changeset': 7679935,
          'id': 7679935,
          'lat': 7679935,
          'lon': 7679935,
          'timestamp': 7679935,
          'uid': 7679934,
          'user': 7679934,
          'version': 7679935},
 'osm': {'attrnum': 1, 'generator': 1, 'version': 1},
 'relation': {'attrnum': 2484,
              'changeset': 2484,
              'id': 2484,
              'timestamp': 2484,
              'uid': 2484,
              'user': 2484,
              'version': 2484},
 'tag': {'attrnum': 6158629, 'k': 6158629, 'v': 6158629},
 'way': {'attrnum': 1087021,
         'changeset': 1087021,
         'id': 1087021,
         'timestamp': 1087021,
         'uid': 1087020,
         'u

#### 统计标签结构化的情况，从而答题了解每个标签结构的组成，对于后续json序列化插入有关键的参考价值

In [4]:
import pprint
import re
import codecs
import xml.etree.cElementTree as ET

##查数据结构，与上述需求不同检查，按照xml结构解析字段统计数
def get_inner_tages(keyDict, keyList):
    if len(keyList) == 1:

        if keyList[0] in keyDict:
            if "Total" in keyDict[keyList[0]]:
                keyDict[keyList[0]]["Total"] += 1
            else:
                keyDict[keyList[0]]["Total"] = 1
        else:
            keyDict[keyList[0]] = {"Total": 1}
    else:
        if keyList[0] not in keyDict:
            keyDict[keyList[0]] = {}
        get_inner_tages(keyDict[keyList[0]], keyList[1:])
    return keyDict


def process_map(filename):
    tags={}
    otheles={}
    for event, element in ET.iterparse(filename):
        if( event =='end'):
            if (element.tag in ["node","way"]):
                alltags = element.findall("tag")
                for tag in alltags:
                    if ((tag.tag == "tag")):
                        key = tag.attrib['k']
                        keySplit = re.split('\:', key)
                        tags = get_inner_tages(tags, keySplit)         
                element.clear()
            else:
                otheles[element.tag]=element.tag
    pprint.pprint(otheles)
    return tags

tags= process_map("chicago.osm")
tags

{'bounds': 'bounds',
 'member': 'member',
 'nd': 'nd',
 'osm': 'osm',
 'relation': 'relation',
 'tag': 'tag'}


{'Andover Court': {'Total': 1},
 'Comment': {'Total': 14},
 'FAA': {'Total': 1},
 'FIXME': {'Total': 274, 'railway': {'Total': 1}, 'ref': {'Total': 33}},
 'LAYER': {'Total': 4},
 'Length': {'Total': 1},
 'Location': {'Total': 1},
 'Montrose Avenue': {'Total': 1},
 'NHD': {'ComID': {'Total': 7675},
  'Elevation': {'Total': 101},
  'FCode': {'Total': 7665},
  'FDate': {'Total': 5230},
  'FTYPE': {'Total': 5227},
  'FType': {'Total': 2429},
  'GNIS_ID': {'Total': 80},
  'GNIS_Name': {'Total': 82},
  'RESOLUTION': {'Total': 5230},
  'ReachCode': {'Total': 6563},
  'way_id': {'Total': 2435}},
 'NHRP': {'Total': 1},
 'NHS': {'Total': 832},
 'NRHP': {'Total': 1},
 'TODO': {'Total': 3},
 'Train': {'Total': 1},
 'abandoned': {'Total': 25,
  'aeroway': {'Total': 1},
  'highway': {'Total': 19},
  'name': {'Total': 8}},
 'access': {'Total': 4232, 'conditional': {'Total': 1}},
 'add': {'city': {'Total': 1}},
 'addr': {'STE': {'Total': 1},
  'Total': 1,
  'city': {'Total': 20150},
  'country': {'Tot

### 替换缩写变成标准单词
addr:street记录了街道名称，其中出现了各类有个人色彩的缩写，因此通过先输出一次后得出所有addr:street名称结果，并针对结果进行整理后更新，将所有收集到的缩写更新为标准词组

In [5]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# 经过测试 有些字段可以直接判定不需要进行更新的，单独列出
exception=["East","Damen","Belmont","US-6","Ter","Wabansia","O"]

def convert_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if (street_type not in expected) and (street_type not in exception):
            street_types[street_type].add(street_name)


def audit(osmfile):
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("end",)):
#   for event, elem in ET.iterparse(osmfile, events=("start",)):
#       if (elem.tag == "node" or elem.tag == "way"):
#            for tag in elem.iter("tag"):
#                if (tag.attrib['k'] == "addr:street"):
#                    convert_street_type(street_types, tag.attrib['v'])
#        elem.clear()

        if (elem.tag in ["node","way"]):
                alltags = elem.findall("tag")  ##效果与.iter效果相同，但一种方式采用end结束处理，另外一种方式采用start方式处理
                for tag in alltags:
                    if ((tag.tag == "tag")):
                        if (tag.attrib['k'] == "addr:street"):
                            convert_street_type(street_types, tag.attrib['v']) 
                elem.clear()
       

    return street_types

def test():
    st_types = audit('chicago.osm')
    pprint.pprint(dict(st_types))
    print("----------------------------------------------------------")
    
if __name__ == '__main__':
    test()

{'104': {'North Sherman Avenue #104'},
 '1220': {'8 S Michigan Ave Suite 1220'},
 '129': {'North Milwaukee Avenue 129'},
 '14': {'Route 14'},
 '20': {'US 20'},
 '231': {'East US 231'},
 '30': {'US 30'},
 '34': {'West Route 34', 'US 34'},
 '38': {'Route 38'},
 '47': {'HWY 47'},
 '53': {'Route 53'},
 '59': {'HWY 59',
        'Route 59',
        'Route IL 59',
        'S HWY 59',
        'S Rte 59',
        'South Route 59'},
 '60008': {'60008'},
 '60435': {'1050 Essington Rd. Joliet, IL 60435'},
 '60463': {'60463'},
 '60546': {'60546'},
 '64': {'Route 64'},
 '83': {'South Route 83'},
 'AV': {'South Harding AV'},
 'AVE': {'S EWING AVE'},
 'Access': {'North Breakwater Access'},
 'Armitage': {'W Armitage'},
 'Ashland': {'South Ashland'},
 'Ave': {'Belmont Ave',
         'Brook Forest Ave',
         'Central Ave',
         'Cicero Ave',
         'Dundee Ave',
         'E Park Ave',
         'East 79th Ave',
         'East Ogden Ave',
         'Forest Ave',
         'Jefferson Ave',
         

In [6]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# 经过测试 有些字段可以直接判定不需要进行更新的，单独列出
exception=["East","Damen","Belmont","US-6","Ter","Wabansia","O"]

def convert_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if (street_type not in expected) and (street_type not in exception):
            street_types[street_type].add(street_name)
            
            
def audit(osmfile):
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("end",)):
#   for event, elem in ET.iterparse(osmfile, events=("start",)):
#       if (elem.tag == "node" or elem.tag == "way"):
#            for tag in elem.iter("tag"):
#                if (tag.attrib['k'] == "addr:street"):
#                    convert_street_type(street_types, tag.attrib['v'])
#        elem.clear()

        if (elem.tag in ["node","way"]):
                alltags = elem.findall("tag")  ##效果与.iter效果相同，但一种方式采用end结束处理，另外一种方式采用start方式处理
                for tag in alltags:
                    if ((tag.tag == "tag")):
                        if (tag.attrib['k'] == "addr:street"):
                            convert_street_type(street_types, tag.attrib['v']) 
                elem.clear()
       

    return street_types


#根据上一条操作的输出记录，展开替换所有已知缩写
mapping = { "St": "Street", "st": "Street",
            "St.": "Street",
            "Ave": "Avenue",
           "Ave.": "Avenue",
           "AVE.": "Avenue",
           "Dr": "drive",
           "Dr.": "drive",
           "Blvd": "boulevard",
           "Blvd.": "boulevard","blvd": "boulevard",
           "Fwy": "freeway",
           "Ln": "lane",
           "Wy": "way",
           "LN": "lane",
           "PKWY": "PARKWAY",
           "Pkwy": "PARKWAY","pkwy": "PARKWAY",
           "Rd": "Road","rd": "Road",
            "Rd.": "Road",
           "Cir":"circle",
           "CT":"court",
           "Ct.":"court",
           "Ct":"court",
           "Ctr.":"court","PLZ":"Plaza",
           "Hwy":"Highway",
           "HWY":"Highway",
           "Hwy.":"Highway"           
            }



def update_name(name, mapping):
    if(len(street_type_re.findall(name))>0):
        if street_type_re.findall(name)[0] in exception:
            return name
        else:
            substring=street_type_re.findall(name)[0]
            if(substring in mapping ):
                newname = street_type_re.sub(mapping[substring], name, 1 )
                return newname
            else:
                return name
    else:
        return name


def run():
    st_types = audit('chicago.osm')
    for st_type, ways in st_types.items():
            for name in ways:
                updatename = update_name(name, mapping)
                pprint.pprint(updatename)
            

if __name__ == '__main__':
    run()

'N. Northwest Highway'
'South Avenue D'
'Elk Grove Town court'
'Touhy Avenue'
'N Damen Avenue'
'Brookfield Avenue'
'Calumet Avenue'
'W. Euclid Avenue'
'N. Oriole Avenue'
'Burlington Avenue'
'Milwaukee Avenue'
'S. Doty Avenue'
'N. Kilbourn Avenue'
'North Clybourn Avenue'
'Grand Avenue'
'W Park Avenue'
'South Avenue M'
'S. Rohlwing Road'
'US 30'
'Center PARKWAY'
'Merchandise Mart Plaza'
'West Merchandise Mart Plaza'
'North Riverside Plaza'
'South Riverside Plaza'
'S EWING AVE'
'Gloucester Way North'
'Parkway North'
'South Chicago'
'Raymond drive'
'Boulder drive'
'Arrowhead drive'
'Varsity drive'
'Gregory M Sears drive'
'Augusta drive'
'Fox Valley Center drive'
'Woodridge drive'
'Summit drive'
'Academic drive'
'Meadows drive'
'Steamboat drive'
'Breckenridge drive'
'Greenbriar drive'
'John M Boor drive'
'Briarwood drive'
'Plaza drive'
'Marketview drive'
'Industrial drive'
'Charlestown drive'
'Regent drive'
'Brentwood drive'
'Copper Mountain drive'
'N Columbus Service drive'
'Alpine drive'


#### 将编辑替换以后的数据写入json文件并写入mongodb

In [7]:
import pdb
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
from pymongo import MongoClient

'''
<node changeset="7781188" id="219850" lat="41.7585879" lon="-87.9101245" timestamp="2011-04-06T05:17:15Z" uid="207745" user="NE2" version="54">
    <tag k="exit_to" v="Joliet Road" />
    <tag k="highway" v="motorway_junction" />
    <tag k="ref" v="276C" />
  </node>
'''

crtinfo = [ "version", "user", "timestamp", "uid", "changeset"]

problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def  parseJson(element):
    node = {}
    if (element.tag in ["node","way","relation"]) :
        #print("parsejson: ",ET.tostring(element).decode())
        createdinfo = {} 
        geo = [] 
        node_refs = [] 
        
        members=[]
        
        node["id"] = element.attrib["id"]
        node["tagtype"] = element.tag
        
    
        try:
            geo.append(float(element.attrib["lat"]))
            geo.append(float(element.attrib["lon"]))
            node["geo"] = geo      
        except KeyError:
            pass
                
        for ck in crtinfo:
            if(ck in element.keys()):
                createdinfo[ck] = element.attrib[ck]  
        node["ndinfo"] = createdinfo
        
        
        for tag in element.iter("tag"): 
            #print(ET.tostring(tag))            
            key = tag.attrib["k"]    #k值存在多端切分:，需要分别保存
            value = tag.attrib["v"]
            arr = re.split('\:', key)
            node = parse_key(node, arr, value)

            try: 
                node["address"] = node.pop("addr")
            except KeyError:
                pass
            
        #relation有memeber节点    
        for mb in element.iter("member"): 
            #print(ET.tostring(tag))
            member={}
            for k in mb.attrib:
                member[k]= mb.attrib[k]
            members.append(member)
        node["member"]=members

        
        #way节点一般有nd关联节点
        for nd in element.iter("nd"): 
            node_refs.append(nd.attrib["ref"]) 
        node["node_refs"] = node_refs            
        
        return node
    else:
        return None

def parse_key(keyDict, keyList, keyValue):      
    if len(keyList) == 1:
        if(keyList[0]=='street'):
            keyValue=update_name(keyValue,mapping)
        keyDict[keyList[0]] = keyValue
    else:
        if keyList[0] not in keyDict:
            keyDict[keyList[0]] = {}
        if isinstance(keyDict[keyList[0]], dict):
            keyDict[keyList[0]]=parse_key(keyDict[keyList[0]], keyList[1:], keyValue)
        else:
            k = ""
            for key in keyList:
                k += key + "_"
            k = k[:-1] 
            keyDict[k] = keyValue
    return keyDict

 
### 经过尝试数据量过大，为了避免内存占用，通过直接传入数据库参数，在业务代码中直接插库，
### fmt支持输出保存数据格式化
def process_map(db,file_in, fmt = False):
    

    file_out = "{0}.json".format(file_in)
    data = []
    idx=0
    with codecs.open(file_out, "w", "utf-8") as fo:
        for event, elem in ET.iterparse(file_in, events=("start","end")):    #针对事件因为需要在内部展开子节点，因此需要events包括开口和闭口
            if(event=="start"):
                
                el = parseJson(elem)          
                idx+=1
                if el:   
                    
                    if fmt:
                        fo.write(json.dumps(el, indent=2)+",\n")
                    else:
                        fo.write(json.dumps(el) + "\n")
                    if("way" in elem.attrib):
                        pprint.pprint(el)
                    db.chicago.insert_one(el)
            elif(event=="end"):
                elem.clear()      
                
    print("all element ",idx)                
    return;


def runInsert():
   
    client = MongoClient("mongodb://192.168.3.155:27017")
    db = client.map
    process_map(db,'chicago.osm', True)
    

runInsert()
    

all element  24043776


In [8]:
import gc
gc.collect()

4356

In [9]:
import pdb
import string 
import pprint
from pymongo import MongoClient
import os
import re

## 基本数据概览

数据源采用芝加哥城市数据进行分析，参考文件要求进行如下选择。

1、数据文件大小 
数据文件大小在1.8GB
采用大型文本数据原因是为了检查使用python解析xml文件的功能理解，在数据准备及插入数据库过程中，基本流程内保证内存占用<800M

2、本次数据清洗后入库有效数据9869440条。其中relation类型数据2484条，way类型数据1087021条，node类型数据7879935条。



In [10]:

    
#query()

pprint.pprint("chicago.osm file size : "+ str(os.path.getsize('chicago.osm')))

client =MongoClient("mongodb://192.168.3.155:27017")
db = client.map.chicago


pprint.pprint("all rows " + str(db.count()))

rs= db.aggregate([{
            "$group":{"_id":"$tagtype", "count":{"$sum":1}}
        }])
pprint.pprint("all tag types " )
for doc in rs:
    pprint.pprint(doc)

'chicago.osm file size : 1887427618'
'all rows 8769440'
'all tag types '
{'_id': 'relation', 'count': 2484}
{'_id': 'way', 'count': 1087021}
{'_id': 'node', 'count': 7679935}


## 数据统计维度分析


### 根据用户提交情况统计，展示哪些用户贡献度最高。按照贡献从多到少提供

In [11]:

print("---------------------------------根据ndinfo.user统计设施次数----------------------------------------------")    
rs= db.aggregate([{
            "$group":{"_id":"$ndinfo.user", "count":{"$sum":1}}},{"$sort":{"count":-1}}
    ])

for doc in rs:
    pprint.pprint(doc)
    

---------------------------------根据ndinfo.user统计设施次数----------------------------------------------
{'_id': 'chicago-buildings', 'count': 5724631}
{'_id': 'Umbugbene', 'count': 1031161}
{'_id': 'alexrudd (NHD)', 'count': 269298}
{'_id': 'woodpeck_fixbot', 'count': 261575}
{'_id': 'TIGERcnl', 'count': 119770}
{'_id': 'patester24', 'count': 115641}
{'_id': 'mpinnau', 'count': 107196}
{'_id': 'bbmiller', 'count': 85927}
{'_id': 'NE2', 'count': 82519}
{'_id': 'Sundance', 'count': 78046}
{'_id': 'Kristian M Zoerhoff', 'count': 67888}
{'_id': 'Tom Layo', 'count': 63017}
{'_id': 'nickvet419', 'count': 58955}
{'_id': 'Deo Favente', 'count': 50629}
{'_id': 'bot-mode', 'count': 48483}
{'_id': 'iandees', 'count': 42925}
{'_id': 'alexrudd', 'count': 34574}
{'_id': 'Rub21', 'count': 33913}
{'_id': 'WernerP', 'count': 31840}
{'_id': 'mappy123', 'count': 24876}
{'_id': 'Eliyak', 'count': 18284}
{'_id': '42429', 'count': 15487}
{'_id': 'boeleman81', 'count': 14555}
{'_id': 'Ru55Ht', 'count': 12731}
{'_

### 按照 amenity  设施分类查看提交了多少种设施。由于插入数据中有多种类型，因此可看到没有amenity标签属性的数据最多，其次是公园6881次， place_of_worship理解为教堂等4314次，学校3389次。

In [12]:

print("---------------------------------根据amenity统计设施次数----------------------------------------------")    
rs= db.aggregate([{
            "$group":{"_id":"$amenity", "count":{"$sum":1}}},{"$sort":{"count":-1}}
    ])

for doc in rs:
    pprint.pprint(doc)

---------------------------------根据amenity统计设施次数----------------------------------------------
{'_id': None, 'count': 8748323}
{'_id': 'parking', 'count': 6881}
{'_id': 'place_of_worship', 'count': 4314}
{'_id': 'school', 'count': 3389}
{'_id': 'restaurant', 'count': 919}
{'_id': 'fast_food', 'count': 770}
{'_id': 'fuel', 'count': 452}
{'_id': 'grave_yard', 'count': 430}
{'_id': 'bicycle_rental', 'count': 299}
{'_id': 'bank', 'count': 276}
{'_id': 'fire_station', 'count': 266}
{'_id': 'cafe', 'count': 235}
{'_id': 'library', 'count': 233}
{'_id': 'shelter', 'count': 190}
{'_id': 'fountain', 'count': 186}
{'_id': 'swimming_pool', 'count': 184}
{'_id': 'post_office', 'count': 180}
{'_id': 'drinking_water', 'count': 179}
{'_id': 'pharmacy', 'count': 173}
{'_id': 'hospital', 'count': 151}
{'_id': 'toilets', 'count': 135}
{'_id': 'pub', 'count': 119}
{'_id': 'bar', 'count': 112}
{'_id': 'post_box', 'count': 91}
{'_id': 'police', 'count': 81}
{'_id': 'townhall', 'count': 76}
{'_id': 'bench',

### 按照 amenity  =school 提交过的全部学校名称，大部分学校都是唯一提交，有些学校被多个用户或者多次提交数据中。芝加哥全部的学校不区分小初高共有2839所，名称中有middle默认为中学，共有140所。其他学校经对数据抽样常看没有特定规律能够识别是小学或者高中

In [13]:

print("---------------------------------根据amenity=school统计设施次数----------------------------------------------")    


rs= db.aggregate([
            {"$match":{"amenity":"school"}},
            {"$group":{"_id":"$name","count":{"$sum":1}}},
            {"$sort":{"count":-1}}
    ])

cnt=0
for doc in rs:
    pprint.pprint(doc)
    cnt=cnt+1

print("共有学校 "+str(cnt) +" 所") 


rs= db.aggregate([
            {"$match":{"amenity":"school","name":re.compile("middle",re.IGNORECASE)}},
            {"$group":{"_id":"$name","count":{"$sum":1}}},
            {"$sort":{"count":-1}}
    ])

cnt=0
for doc in rs:
    pprint.pprint(doc)
    cnt+=1

print("中学共有 "+str(cnt)+" 所")


---------------------------------根据amenity=school统计设施次数----------------------------------------------
{'_id': None, 'count': 166}
{'_id': 'Lincoln Elementary School', 'count': 16}
{'_id': 'Lincoln School', 'count': 11}
{'_id': 'Saint Marys School', 'count': 10}
{'_id': 'Saint Joseph School', 'count': 9}
{'_id': 'Sacred Heart School', 'count': 8}
{'_id': 'Saint Mary School', 'count': 8}
{'_id': 'Saint Johns School', 'count': 8}
{'_id': 'Central School', 'count': 8}
{'_id': 'Washington Elementary School', 'count': 8}
{'_id': 'Immaculate Conception School', 'count': 7}
{'_id': 'Washington School', 'count': 6}
{'_id': 'Whittier Elementary School', 'count': 6}
{'_id': 'Trinity School', 'count': 6}
{'_id': 'Immanuel School', 'count': 6}
{'_id': 'Central Elementary School', 'count': 6}
{'_id': 'Saint Paul School', 'count': 5}
{'_id': 'Jefferson Elementary School', 'count': 5}
{'_id': 'Benjamin Franklin Elementary School', 'count': 5}
{'_id': 'Liberty Elementary School', 'count': 5}
{'_id': 'L

middle学校数据经由以下24名用户提交，最多提交次数83次，大部分都提交1个条。数据贡献集中在'iandees' 与 'Umbugbene'.针对用户iandees进行分析，得知该用户在各类数据节点都有贡献，贡献度最高的是完善了39483个node节点数据，并且提交了1925条学校信息和自行车出租(bicycle_rental)信息。以上三项是该用户在芝加哥开放数据中贡献最多的部分

In [14]:

print("---------------------------------根据amenity=school,统计提交人情况统计设施次数----------------------------------------------")    


rs= db.aggregate([
            {"$match":{"amenity":"school","name":re.compile("middle",re.IGNORECASE)}},
            {"$group":{"_id":"$ndinfo.user","count":{"$sum":1}}},
            {"$sort":{"count":-1}}
    ])

cnt=0
for doc in rs:
    pprint.pprint(doc)
    cnt+=1

print("共有 "+str(cnt)+" 名用户提交")




---------------------------------根据amenity=school,统计提交人情况统计设施次数----------------------------------------------
{'_id': 'iandees', 'count': 83}
{'_id': 'Umbugbene', 'count': 29}
{'_id': 'mpinnau', 'count': 5}
{'_id': 'Kristian M Zoerhoff', 'count': 5}
{'_id': 'Sundance', 'count': 3}
{'_id': 'bbmiller', 'count': 1}
{'_id': 'GXCEB0TOSM', 'count': 1}
{'_id': 'TMLutas', 'count': 1}
{'_id': 'Drew2324', 'count': 1}
{'_id': 'OSMF Redaction Account', 'count': 1}
{'_id': 'JaapK', 'count': 1}
{'_id': 'Eliyak', 'count': 1}
{'_id': 'Steven Vance', 'count': 1}
{'_id': 'mappy123', 'count': 1}
{'_id': 'Geo_Liz', 'count': 1}
{'_id': 'patester24', 'count': 1}
{'_id': '42429', 'count': 1}
{'_id': 'BeatlesZeppelinRush', 'count': 1}
{'_id': 'DaBunny', 'count': 1}
{'_id': 'lgoughenour', 'count': 1}
{'_id': 'Thyais Meade', 'count': 1}
{'_id': 'JWAddison', 'count': 1}
{'_id': 'jbk123', 'count': 1}
{'_id': 'Forest Gregg', 'count': 1}
共有 24 名用户提交


In [15]:

print("---------------------------------根据user=iandees,统计提交人情况统计设施次数----------------------------------------------")    


rs= db.aggregate([
            {"$match":{"ndinfo.user":"iandees"}},
            {"$group":{"_id":"$tagtype","count":{"$sum":1}}},
            {"$sort":{"count":-1}}
    ])

cnt=0
for doc in rs:
    pprint.pprint(doc)
    cnt+=1

print("iandees在 "+str(cnt)+" 类节点提交数据\r\n\r\n")

rs= db.aggregate([
            {"$match":{"ndinfo.user":"iandees"}},
            {"$group":{"_id":"$amenity","count":{"$sum":1}}},
            {"$sort":{"count":-1}}
    ])

cnt=0
for doc in rs:
    pprint.pprint(doc)
    cnt+=1

print("iandees在 "+str(cnt)+" 种amenity设施种类下提交")

---------------------------------根据user=iandees,统计提交人情况统计设施次数----------------------------------------------
{'_id': 'node', 'count': 39483}
{'_id': 'way', 'count': 3435}
{'_id': 'relation', 'count': 7}
iandees在 3 类节点提交数据


{'_id': None, 'count': 39683}
{'_id': 'school', 'count': 1925}
{'_id': 'bicycle_rental', 'count': 298}
{'_id': 'grave_yard', 'count': 206}
{'_id': 'restaurant', 'count': 153}
{'_id': 'fast_food', 'count': 151}
{'_id': 'place_of_worship', 'count': 121}
{'_id': 'post_office', 'count': 87}
{'_id': 'parking', 'count': 72}
{'_id': 'hospital', 'count': 61}
{'_id': 'cafe', 'count': 26}
{'_id': 'bank', 'count': 21}
{'_id': 'pub', 'count': 20}
{'_id': 'fuel', 'count': 19}
{'_id': 'pharmacy', 'count': 10}
{'_id': 'bicycle_parking', 'count': 8}
{'_id': 'post_box', 'count': 7}
{'_id': 'bar', 'count': 6}
{'_id': 'gym', 'count': 5}
{'_id': 'bench', 'count': 5}
{'_id': 'parking_entrance', 'count': 4}
{'_id': 'fire_station', 'count': 3}
{'_id': 'car_wash', 'count': 3}
{'_id': 'theat

#### 业务分析实现后的数据建议
数据格式相对固定，但是数据源还是存在很多不规范的缩写，随意性很强。比如addr.CT,Rd,rd类似的大小写区分不规范，在数据使用之前需要进行手工处理重新归类。数据分类使用: 作为分隔符感觉不是很合理，既然xml文件是否更应该充分利用xml的格式特定进行处理，虽然层次多，但可以做到更明显。过去一般都针对几十MB的xml进行全文档树结构加载，因此局限性很大。本次试用iterparse效果远比全树加载好。
xml文件在使用过程中对详细的字段应用方式没有很明确的说明，基本是参考相关案例和项目练习内容进行的编写，并且在不断试错中进行修改猜得到相对完整的数据拆分方法。但依旧不确认所进行的数据清洗是否已全面覆盖必要数据项

#### 建议及预期效果
##### 建议1
数据单位统一，数据变量(tag)规范化
所有数据格式应当统一，各类标签内的字符串应当合理使用相同的缩写，而不能自成一套
数据单位需要统一，虽然本次统计数据没有涉及到太多数值类参数，但根据相关项目和案例介绍，数值类数据中由于缺少约束，一定会出现"800"和"800m"并存，zipcode出现非数字字母的情况。因此在数据采集录入时，建议针对相关字段进行数据约束，保存之前检查数据合法性，如联系电话符合当地区号-电话长度等。缩写识别后给出统一代替或者修改

##### 建议二
单个数据文件中贡献度的差异
众人拾柴火焰高，目前可见很多数据的提交集中在某一些人或者某几个人身上，这种现象表达出openstreetmap中数据来源单一性，是否可能集中表现了几位义务贡献者的意志，以及很多详细的更多维度的数据描述还没有人提供。我相信这个标准四海内皆准。中国地图之前我下载过，单个文件远没有芝加哥城这么大，印象中之前尝试过下载香港中文数据，以降低数据理解难度，但后续由于找不到源，还是采用了一开始下载的芝加哥全数据。而香港地图大小貌似也大于北京地图，而实际情况来看，北京市全市数据内容理论上会接近芝加哥城市或至少大于香港地图数据。如何推动提升贡献者活跃度以及提供何种便利的数据采集方式是将来会面临的问题。比如在当前发展环境下，是否可以采用手机数据收集、实时反馈的形式进行标记，利用国内微信、国外line等工具的LBS特性由用户主动方便的提交所在地点数据集，通过这种方式能够迅速补充各国各地的相关数据。目前我了解到的Mapsme很有可能采用的就是openstreetmap地图。如果能够得到更多的数据补充，作为旅游者手中相对方便的离线地图更有益处，或者说通过这种旅游工具让更多游客自发上传数据内容

#### 项目参考说明
历史学生完成的优秀案例

https://github.com/baumanab/udacity_mongo_github/tree/master/final_project

数据内存占用的性能评价

https://pycoders-weekly-chinese.readthedocs.io/en/latest/issue6/processing-xml-in-python-with-element-tree.html

数据处理查询的技能点,包括mongo,xml处理形式

https://stackoverflow.com/search?q=python+pymongo
https://stackoverflow.com/questions/tagged/python+xml+et